In [ ]:
# default_exp tasks

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
from biocores.softwares.default import *
from biocores.softwares.bwa import Bwa
from biocores.softwares.bowtie import Bowtie
from biocores.softwares.bowtie2 import Bowtie2
from create_reference.commands import *
from create_reference.utils import file_exists

In [ ]:
# export

def task_get_file(link,filename):
    if file_exists(filename):
        return 'echo {filename} exists'.format(filename=filename)
    else:
        cmds = [
            cmd_mkdir(os.path.dirname(filename)),
            cmd_wget(link,filename)
        ]
        return '\n'.join(cmds)

def task_bwa_build_index(software,reference):
    bwa = Bwa(software,bwaDefault)
    cmds = [
            bwa.cmd_build_index(reference)
        ]
        
    return '\n'.join(cmds)

def task_bowtie_build_index(software,reference, prefix):
    bowtie=Bowtie(software,bowtieDefault)
    cmds = [
            cmd_mkdir(os.path.dirname(prefix)),
            bowtie.cmd_build_index(reference, prefix)
        ]
        
    return '\n'.join(cmds) 

def task_bowtie2_build_index(software,reference, prefix):
    bowtie2=Bowtie2(software,bowtie2Default)
    cmds = [
            cmd_mkdir(os.path.dirname(prefix)),
            bowtie2.cmd_build_index(reference, prefix)
        ]
        
    return '\n'.join(cmds) 

In [ ]:
bwa='bwa'
bowtie='bowtie'
bowtie2='bowtie2'
reference = 'genome.fasta'
bowtie_prefix = '/tmp/bowtie_index'
bowtie2_prefix='/tmp/bowtie2_index'
print(task_bwa_build_index(bwa,reference))
print(task_bowtie_build_index(bowtie,reference,bowtie_prefix))
print(task_bowtie2_build_index(bowtie2,reference,bowtie2_prefix))


bwa  index  genome.fasta
mkdir -p /tmp
bowtie-build    genome.fasta /tmp/bowtie_index
mkdir -p /tmp
bowtie2-build    genome.fasta /tmp/bowtie2_index


In [ ]:
hs_link='ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
ss_link='ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz'
outdir='/tmp/'
species='homo_sapiens'
version='99'
filename='genome.fasta.gz'


In [ ]:
print(task_get_file(hs_link,filename))

echo No need to make dirs
wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -O genome.fasta.gz


In [ ]:
print(task_get_file(ss_link,filename))

echo No need to make dirs
wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz -O genome.fasta.gz


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_defaults.ipynb.
Converted 02_tasks.ipynb.
Converted 03_commands.ipynb.
Converted 04_pipelines.ipynb.
Converted 05_recipes.ipynb.
Converted index.ipynb.
